# Instrument Pricing Analytics - Fixed Rate Bond Pricing

Bond pricing was one of the earlier APIs available as part of the IPA offering on our Refinitiv Data Platform.  

To illustrate just how easy it is to use the APIs, I will show some examples around pricing:
* Fixed Rate Bonds
* Floating Rate Notes

### Refinitiv Workspace Calculators

**Fixed Rate Bond Calculator**

![Fixed Rate Bond Calc](FBRCalc250221.png)

Uses the Instrument Pricing Analytics APIs for its computations.

#### Replicate Desktop Calculations with Python code

Let's go ahead and explore some Calculator examples along with the equivalent Python code snippets.

#### Initialisation


In [1]:
import refinitiv.dataplatform as rdp
from refinitiv.dataplatform.content import ipa
from refinitiv.dataplatform.content.ipa import bond
import json
import configparser as cp
from IPython.display import HTML
import pandas as pd

RDP Library is currently available in 
* Python 
* .NET
* TypeScript (soon)

RDP APIs are implemented as RESTful APIs -  use other languages to HTTP POST the request &  parse the Response.  

Using RDP libraries? Saves some overhead - authentication, building Request message, parsing Response etc.

#### Connect to the Refintiv Data Platform

Platform session or Desktop Session


In [2]:
config = cp.ConfigParser()
config.read("config.cfg")

['config.cfg']

In [3]:
#rdp.open_desktop_session(config['desktop']['app_key'])

##### OR

In [4]:
session = rdp.open_platform_session(
    config['session']['app_key'],
    rdp.GrantPassword( username=config['session']['user'], password=config['session']['password'] )
)

### Example 1 : Get Price and Proceeds


#### RDP Python Library Function Layer
The Python Refinitiv Data Platform library offers three interface layers:
1. Function: Highest level - single function call (most popular RDP APIs)
2. Content: High level - Similar to Function layer with extended response (+ Level 1 Streaming data)
3. Delivery: RDP APIs not yet supported by above Layers (+ Level 2 Streaming data)

Function layer offers support for most popular IPA content - including Bond Analytics. 

In [5]:
rdp.get_bond_analytics(
    universe = [
        "US949763RZ13"
    ],
    calculation_params = bond.CalculationParams(
        market_data_date="2021-02-25",
        price_side = ipa.enum_types.PriceSide.BID
    ),
    fields = [
        "RIC",
        "Price",
        "YieldPercent",
        "YieldType",
        "AccruedDays",
        "Accrued",
        "DirtyPrice",
        "CleanMarketValueInDealCcy",
        "AccruedAmountInDealCcy",
        "MarketValueInDealCcy",
        "ErrorMessage"
    ]
)

,RIC,Price,YieldPercent,YieldType,AccruedDays,Accrued,DirtyPrice,CleanMarketValueInDealCcy,AccruedAmountInDealCcy,MarketValueInDealCcy,ErrorMessage
0,949763RZ1=,107.1228,0.250584,Native,27,0.24411,107.36691,1071228,2441.09589,1073669.09589,



Function layer returns Pandas dataframe


Content not supported by Function layer OR not using the RDP Library? - **RESTful interface**

RDP Library helper class for RESTful interface

#### Identifying the Financial Contracts Endpoint
Ascertain the Endpoint, using the Refinitiv Data Platform's <a href="http://api.refinitiv.com/" target="_blank">API Playground</a> 

Search for say *'contracts'* and select the relevant Endpoint. 

Copy Endpoint URL e.g. - ***/data/quantitative-analytics/v1/financial-contracts***

#### Define our API Endpoint for the Fincancial Contracts API

In [6]:
financial_contract_api = rdp.Endpoint(session, 
            'https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts')

#### Format and send the JSON request

Instead passing parameters and fields into the function - construct a JSON message with the similar values - changing the pricing assumptions.  
Submit JSON request to API using Endpoint - can do the same with a HTTP Post request

In [7]:
request_body={
    "fields": [
        "MarketDataDate",
        "ValuationDate",
        "RIC",
        "CouponRatePercent",
        "RedemptionDate",
        "PriceSide",
        "CleanPrice",
        "DirtyPrice",    
        "Accrued",
        "YieldPercent"
    ],
    "outputs": [
        "Data",
        "Headers"
    ],
    "universe": [
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "instrumentCode": "US949763RZ13"
            },
            "pricingParameters": {
                "marketDataDate": "2021-02-25",
                "yieldPercent": 0.21
            }
        }
    ]
}

response = financial_contract_api.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)
print(json.dumps(response.data.raw, indent=2))

{
  "headers": [
    {
      "type": "DateTime",
      "name": "MarketDataDate"
    },
    {
      "type": "DateTime",
      "name": "ValuationDate"
    },
    {
      "type": "String",
      "name": "RIC"
    },
    {
      "type": "Float",
      "name": "CouponRatePercent"
    },
    {
      "type": "DateTime",
      "name": "RedemptionDate"
    },
    {
      "type": "String",
      "name": "PriceSide"
    },
    {
      "type": "Float",
      "name": "CleanPrice"
    },
    {
      "type": "Float",
      "name": "DirtyPrice"
    },
    {
      "type": "Float",
      "name": "Accrued"
    },
    {
      "type": "Float",
      "name": "YieldPercent"
    }
  ],
  "data": [
    [
      "2021-02-25T00:00:00Z",
      "2021-02-25T00:00:00Z",
      "949763RZ1=",
      3.3,
      "2023-06-29T00:00:00Z",
      "Mid",
      107.221,
      107.465109589041,
      0.244109589041096,
      0.21
    ]
  ]
}


Response is JSON payload - rather than DataFrame returned by Function call version  
Optionally requested **Headers** in my request - DataFrame with headings (if required):

In [8]:
pd.DataFrame(data=response.data.raw['data'], columns=[h['name'] for h in response.data.raw['headers']])

,MarketDataDate,ValuationDate,RIC,CouponRatePercent,RedemptionDate,PriceSide,CleanPrice,DirtyPrice,Accrued,YieldPercent
0,2021-02-25T00:00:00Z,2021-02-25T00:00:00Z,949763RZ1=,3.3,2023-06-29T00:00:00Z,Mid,107.221,107.46511,0.24411,0.21


For the remaining examples, I will use the Endpoint interface:
* easier to follow for using RESTful interface 
* access content not yet supported by Function layer

### Example 2: Compute Cash Flows
Another use of the Financial Contracts API is to Compute Cash flows e.g.:

In [9]:
request_body={
    "fields": [
        "CashFlowDatesArray",
        "CashFlowAnnualRatesArray",
        "CashFlowInterestPercentsArray",
        "CashFlowInterestAmountsInDealCcyArray",
        "CashFlowCapitalAmountsInDealCcyArray"
        #"CashFlowTotalPercentsArray",
        #"CashFlowTotalAmountsInDealCcyArray",
        #"CashFlowNormalizationFactorsArray",
        #"CashFlowResidualAmountsInDealCcyArray",
        #"CashFlowCapitalPercentsArray",
    ],
    "outputs": [
        "Data", "Headers"
    ],
    "universe": [
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "instrumentCode": "US949763RZ13"
            },
            "pricingParameters": {
                "marketDataDate": "2021-02-26",
                "priceSide": "Bid"
            }
        }
    ]
}

response = financial_contract_api.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)
df = pd.DataFrame(data=[item for elem in response.data.raw['data'] for item in elem]).transpose()
df.columns = [h['name'] for h in response.data.raw['headers']]
df

,CashFlowDatesArray,CashFlowAnnualRatesArray,CashFlowInterestPercentsArray,CashFlowInterestAmountsInDealCcyArray,CashFlowCapitalAmountsInDealCcyArray
0,2021-03-01T00:00:00Z,3.3,0.275,2750.0,0.0
1,2021-03-29T00:00:00Z,3.3,0.275,2750.0,0.0
2,2021-04-29T00:00:00Z,3.3,0.275,2750.0,0.0
3,2021-06-01T00:00:00Z,3.3,0.275,2750.0,0.0
4,2021-06-29T00:00:00Z,3.3,0.275,2750.0,0.0
5,2021-07-29T00:00:00Z,3.3,0.275,2750.0,0.0
6,2021-08-30T00:00:00Z,3.3,0.275,2750.0,0.0
7,2021-09-29T00:00:00Z,3.3,0.275,2750.0,0.0
8,2021-10-29T00:00:00Z,3.3,0.275,2750.0,0.0
9,2021-11-29T00:00:00Z,3.3,0.275,2750.0,0.0


Just one example of the outputs - many other fields available


### Example 3: Computing Spreads



Spreads can be calculated with reference to various Benchmarks such as
* government benchmark that should be computed for all bonds
* government country benchmark e.g. for bonds of European countries
* issuer benchmark for corporate bonds
* EDSF (EuroDollar) benchmark -  US bonds with low maturity
* User Defined benchmark - input ric has defined by the user    
and so on..

Each benchmark can be overriden in terms of Yield, Spread basis etc.




In [10]:
request_body={
     "outputs":["Data"],
    "fields": [
        "GovernmentSpreadBp",
        "GovernmentBenchmarkCurvePrice",
        "GovernmentBenchmarkCurveYieldPercent",
        "GovernmentBenchmarkCurveName",
        "GovernmentBenchmarkCurveNearestRicDescription",
        "GovernmentBenchmarkCurveLowerRicDescription",
        "GovernmentBenchmarkCurveUpperRicDescription",
        
        "AssetSwapSpreadBp",
        "UpfrontSpreadAmount",
        "ZSpreadBp",
        "DiscountCurveName"
    ],
    "universe": [
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "instrumentCode": "US949763RZ13"
            },
            "pricingParameters": 
            {
                  "marketDataDate": "2019-09-12",
                  "benchmarkYieldSelectionMode":"Nearest"
              }
        }
    ]
}

response = financial_contract_api.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

print(json.dumps(response.data.raw, indent=2))

{
  "data": [
    [
      143.721984511,
      99.52734375,
      0.408680799,
      "0#USBMK=",
      "US 3Y T-NOTE    ",
      "US 3Y T-NOTE    ",
      "US 5Y T-NOTE    ",
      25.768945614,
      -54754.7534246574,
      25.705763937,
      "USD - Swap SB/3M Libor"
    ]
  ]
}


#### Other computations

Much more we can do with the Financial Contracts API e.g.
* Computing Option Adjusted measures
* Pricing using Custom Redemption dates
* Using a different currency
* Define Custom rounding  
and so on....



In [11]:
session.close()

<State.Closed: 1>